In [1]:
from zlib import crc32
import numpy as np

if __name__=='__main__':
    mail = input('enter phystech e-mail\n') 
    to_hash =  mail.strip().split('@')[0]    
    h = crc32(to_hash.encode('utf-8'))
    seed = h%(2**32-1)
    rs = np.random.RandomState(seed)
    task1 = rs.randint(low=1, high=5)
    task2 = rs.randint(low=1, high=5)
    task3 = rs.randint(low=1, high=5)
    task4 = rs.randint(low=1, high=4)
    print ('Your tasks are 1.{}, 2.{}, 3.{}, 4.{}'.format(task1, task2, task3, task4))

enter phystech e-mail
akimenkova.ma@phystech.edu
Your tasks are 1.2, 2.3, 3.2, 4.3


In [2]:
import scipy.stats as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.datasets import load_iris
import statsmodels.stats.api as sms
from arch.bootstrap import IndependentSamplesBootstrap

# Задача 3.2

Выборка представляет собой статистику бросков для различных игроков NBA. Для каждого игрока известны:

количество успешных бросков в домашних играх (score_home)

количество бросков в домашних играх (atm_home)

количество успешных бросков в гостевых играх (score_away)

количество бросков в гостевых играх (atm_away)

Требуется определить, есть ли разница в успехе бросков у игроков в домашних и гостевых играх.

У какого процента игроков разница в успехе существенна?

In [4]:
!wget 'https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad-2020/master/lab/lab1/data/nba.csv' -O 'nba.csv'

--2020-03-25 19:39:49--  https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad-2020/master/lab/lab1/data/nba.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.84.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.84.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34909 (34K) [text/plain]
Saving to: ‘nba.csv’

nba.csv             100%[===================>]  34,09K  --.-KB/s    in 0,06s   

2020-03-25 19:39:49 (580 KB/s) - ‘nba.csv’ saved [34909/34909]



In [7]:
nba = pd.read_csv('nba.csv', index_col=0)

In [11]:
nba.head()

,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
7,A. Johnson,ATL,30,34,24,28,64,52
10,A. Johnson,TOR,63,57,44,38,120,82
15,Abdur-Rahim,SAC,114,124,84,90,238,174
22,Afflalo,DEN,236,284,189,227,520,416
23,Afflalo,DET,75,88,62,67,163,129


In [41]:
nba.index = nba['player']

In [47]:
nba = nba.drop(['team', 'player'], axis=1)

In [57]:
nba = nba.groupby('player').agg(sum)

In [58]:
nba.head()

,atm_away,atm_home,score_away,score_home,atm_total,score_total
player,,,,,,
A. Johnson,93,91,68,66,184,134
Abdur-Rahim,114,124,84,90,238,174
Afflalo,311,372,251,294,683,545
Aldridge,899,940,678,748,1839,1426
Alexander,36,47,24,34,83,58


In [13]:
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

Без поправок:

In [59]:
p_values = []
for _, player in nba.iterrows():
    p_value = proportions_ztest((player['score_away'], player['score_home']), 
                                (player['atm_away'], player['atm_home']), 
                                alternative='two-sided')[1]
    p_values.append(p_value)

In [60]:
p_values = np.asarray(p_values)

In [61]:
(p_values < 0.05).sum()

32

#### Без поправок отвергается 32 гипотезы

С поправкой:

In [62]:
multipletests(p_values, method='bonferroni')[0].sum()

1

In [63]:
multipletests(p_values, method='holm')[0].sum()

1

Поправки Бонферрони и Холма позволяют отвергнуть 1 гипотезу.